Allstate is an insurance company that pays out claims to customers who get into car accidents. Depending on the severity of the accident, the cost of the insurance claim varies. This competition aims to predict claim severity given a variety of claim features
Our target is ‘loss’, or the dollar amount lost by Allstate due to this claim. There are two types of features in the given data, 116 categorical features and 14 continuous numeric features. Submissions to this competition are evaluated by the mean absolute error (MAE) between the predicted loss and the actual loss. In this notebook, I will be implementing and evaluating several models to try and predict Allstate's claim losses with the highest possible accuracy.


First, the necessary libraries are loaded in.

In [ ]:
#Load in Libraries
library(bonsai)
library(lightgbm)
library(tidymodels)
library(embed) 
library(vroom)
library(tidyverse)

Here, I load in the test and train datasets.

In [ ]:
train_data <- vroom("/kaggle/input/allstate-claims-severity/train.csv")
test_data <- vroom("/kaggle/input/allstate-claims-severity/test.csv")

The tidymodels package in R allows us to create feature-engineered workflows that can be applied to a variety of models. This package will be used throughout this project. In the training data, there is a uniquely identifying ID column that does not lend us any explainability for modeling purposes, so I changed the data type to reflect its role. I also assigned all of the categorical features in the training set as factors and target-encoded them for use in future modeling. I then normalized all of the numeric features on a scale from 0 to 1.

In [ ]:
recipe <- recipe(loss~., data=train_data) %>%
  update_role(id, new_role="id") %>%
  step_mutate_at(all_nominal_predictors(), fn = as.factor) %>%
  step_lencode_glm(all_nominal_predictors(), outcome = vars(loss)) %>% 
  step_range(all_numeric_predictors(), min=0, max=1) #scale to [0,1]

First, a penalized regression model is fit on the training data. I used a penalty of 5 and a mixture of 0.75, which leans the model more towards lasso regularization. The penalized regression model is then fit to the training data.

In [ ]:
#Set model and tuning
preg_model <- linear_reg(penalty=5, mixture=0.75) %>% 
  set_engine("glmnet") 

preg_wf <- workflow() %>%
  add_recipe(recipe) %>%
  add_model(preg_model) %>%
  fit(data=train_data)

Once the model has been fit, predictions are made on the test data and the predictions are submitted to measure the out of sample MAE.

In [ ]:
preg_preds <- predict(preg_wf, new_data=test_data)

kaggle_submission <- preg_preds %>%
  bind_cols(., test_data) %>% #Bind predictions with test data
  select(id, .pred) %>% #Just keep datetime and prediction variables
  rename(loss=.pred)

## Write out the file
vroom_write(x=kaggle_submission, file="./PenalizedRegPreds.csv", delim=",")

The MAE for this penalized regression model was 1291.94, which is alright but can definitely be improved. I'll try some more models to see if we can decrease our score.

Next, a regression tree was fit to the training data. I chose to tune the tree depth, cost complexity, and minimum number of samples until node split hyperparameters. The other hyperparameters were set at their default values.

In [ ]:
tree_mod <- decision_tree(tree_depth = tune(),
                        cost_complexity = tune(),
                        min_n=tune()) %>% 
  set_engine("rpart") %>% 
  set_mode("regression")


tree_wf <- workflow() %>%
  add_recipe(recipe) %>%
  add_model(tree_mod) 

I then performed K-fold cross-validation to tune the hyperparameters, using 3 folds and 9 different hyperparameter combinations. 

In [ ]:
grid_of_tuning_params <- grid_regular(tree_depth(),
                                      cost_complexity(),
                                      min_n(),
                                      levels = 3) ## L^2 total tuning possibilities
## Split data for CV
folds <- vfold_cv(train_data, v = 3, repeats=1)

## Run the CV1
CV_results <- tree_wf %>%
  tune_grid(resamples=folds,
            grid=grid_of_tuning_params,
            metrics=metric_set(mae)) #Or leave metrics NULL

The model with the best performing hyperparameters relative to the resulting MAE was chosen, and the final model was fit using these hyperparameters to the training data.

In [ ]:
## Find Best Tuning Parameters13
bestTune <- CV_results %>%
  select_best(metric ="mae")

final_wf <-
  tree_wf %>%
  finalize_workflow(bestTune) %>%
  fit(data=train_data)

Once the final model was fit, it was used to make predictions on the test data and submitted to kaggle to determine the out-of-sample MAE.

In [ ]:
preds <- predict(final_wf, new_data=test_data)

kaggle_submission <- preds %>%
  bind_cols(., test_data) %>% #Bind predictions with test data
  select(id, .pred) %>% #Just keep datetime and prediction variables
  rename(loss=.pred)

## Write out the file
vroom_write(x=kaggle_submission, file="./TreePreds.csv", delim=",")

The MAE for this model was 1312.74, which was slightly worse than the penalized regression, but I believe my score can still be improved.

Lastly, I chose to fit a boosted tree regression model to the data. There are several packages that can be used to run a boosted tree model, but I have found that Light GBM runs the quickest without decreasing performance. The hyperparameters that will be tuned the depth of the trees, the number of trees, and the learning rate. Other parameters, such as the number of leaves and the minimum number of samples required to split a node, could be tuned, but to avoid overfitting I am setting those at their default values.

In [ ]:
boost_model <- boost_tree(tree_depth=tune(),
                          trees=tune(),
                          learn_rate=tune()) %>%
  set_engine("lightgbm") %>% 
  set_mode("regression")

The workflow is set up with the previously defined recipe and the boosted tree model with the specified hyperparameters to be tuned.

In [ ]:
# Set up the workflow
gbm_wf <- workflow() %>%
  add_recipe(recipe) %>%
  add_model(boost_model)

To tune and measure the performance of our model, I performed k-fold cross-validation with 3 folds. For tuning, I created a regular grid with 9 combinations of hyperparameters. The performance of each hyperparameter combination was measured by mean absolute error (MAE), which is the same performance metric used by the competition.

In [ ]:
cv_folds <- vfold_cv(train_data, v = 3, repeats = 1)

grid <- grid_regular(tree_depth(),
                            trees(),
                            learn_rate(),
                            levels = 3)

tuned_results <- gbm_wf %>% 
    tune_grid(
      resamples = cv_folds,
      grid = grid,
      metrics = metric_set(mae),
      control = control_grid()
    )

The hyperparameter combination with the best performance relative to its MAE was selected, and the boosted model was fit to the training data with these parameters.

In [ ]:
# Get the best hyperparameters
best_params <- tuned_results %>% 
  select_best(metric = "mae")

# Finalize the workflow with the best parameters
final_workflow <- gbm_wf %>%  
    finalize_workflow(best_params) %>%  
    fit(data = train_data)

Now that the highest-performing model has been fit, predictions are made on the test set. Once made, the predictions are bound to the test set, and the predictions are submitted to obtain an out-of-sample MAE. 

In [ ]:
# Make predictions on the test data
boosted_preds <- predict(final_workflow, new_data = test_data)

kaggle_submission <- boosted_preds %>%
  bind_cols(., test_data) %>% #Bind predictions with test data
  select(id, .pred) %>% #Just keep datetime and prediction variables
  rename(loss=.pred)
#rename pred to count (for submission to Kaggle)
vroom_write(x=kaggle_submission, file="./boostedPreds.csv", delim=",")


The out-of-sample MAE for the boosted tree model is 1163.85, which made it the best performing out of all of the models I tried. Therefore, this was the model that I chose to use.